In [1]:
import numpy as np 
import pandas as pd 
import geopandas as gpd
import shapely as shp
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors
from shapely.ops import nearest_points
"""
GLOBAL VARIABLES
"""
COUNTY_SHP_FILE = "data/UScounties/UScounties.shp"
BANNED_STATES = ['Arkansas','Alabama','Idaho','Kentucky','Louisiana','Kentucky','Mississippi','Missouri','Oklahoma','South Dakota','Tenesee','Texas','West Virginia','Wisconsin']

EPSG = 4326

In [2]:

"""load_state_bounds(): given a shp file of county locations, return a GeoDataFrame with the states as rows
    input:
        COUNTY_SHP_FILE<str>: the path to a shp file containing us county data
        EPSG: the ESPG id of the coordinate reference system to use
"""
def load_state_bounds(COUNTY_SHP_FILE,EPSG=4326):
    df_county = gpd.read_file(COUNTY_SHP_FILE)
    df_county = df_county.to_crs(f"EPSG:{EPSG}")
    df_county = df_county[~df_county['STATE_NAME'].isin(['Alaska','Hawaii'])]#remove Alaska and Hawai
    df_state = df_county.dissolve(by = "STATE_NAME")['geometry'].reset_index()#[['STATE_NAME','geometry']]#aggregate by stateI
    return df_state

"""
random_points_within_polygon(); raturns a list of n points within a given polygon
    inputs: 
        polygon<Shapley.geometry.polygon>: a polygon from which to generate points inaside of 
        number<int>: the number of points to generate
    output:
        points<list{Shapley.geometry.Points}: a list of points randomly distributed within the polygon
"""
def random_points_within_polygon(polygon,number):
    points = []
    minx,miny,maxx,maxy = polygon.bounds
    while len(points) <  number:
        pnt = Point(np.random.uniform(minx,maxx),np.random.uniform(miny,maxy))
        if polygon.contains(pnt):
            points.append(pnt)
    return gpd.GeoDataFrame(geometry=points)


def nearest(row,geom_union,df1,df2,geom1_col = 'geometry',geom2_col = 'geometry',src_column = None):
    #find the geometry that is closest
    test = row[geom1_col] 
    print("Hello")
    nearest = df2[geom2_col] == nearest_points(row[geom1_col],geom_union)[1]
    print("neatest_complete")
    value = df2[nearest][src_column].get_values()[0]
    return value

In [3]:
#load county data 
df_pop = pd.read_csv("data/simulated_pop_points.csv")
gdf_pop = gpd.GeoDataFrame(df_pop,geometry = gpd.points_from_xy(df_pop.lon,df_pop.lat))
#load state boundaries
df_state = load_state_bounds(COUNTY_SHP_FILE)
df_state_legal = df_state[~df_state['STATE_NAME'].isin(BANNED_STATES)]
us_border = df_state.dissolve().geometry.values[0]#extract shapley polygons from dataframe
us_legal_border = df_state.dissolve().geometry.values[0]#extract shapley polygon from dataframe

fac_placements_df = random_points_within_polygon(us_border,100).reset_index()

/Users/willthompson/miniconda3/envs/myenv/lib/python3.9/site-packages/geopandas/array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


In [43]:
unary_union = fac_placements_df.unary_union
gdf_pop.apply(nearest,geom_union = unary_union,df1=gdf_pop,df2=fac_placements_df,src_column = 'index',axis=1)

NameError: name 'gdf_pop' is not defined

In [ ]:
fac_placements_df

,geometry
0,POINT (-74.74201 42.49377)
1,POINT (-116.96697 40.69209)
2,POINT (-113.57297 46.65724)
3,POINT (-68.41258 44.51050)
4,POINT (-91.54317 34.18865)
...,...
95,POINT (-96.87855 36.62089)
96,POINT (-80.78350 37.18219)
97,POINT (-97.17643 43.43787)
98,POINT (-81.15291 26.39950)
